# 0. Installing Transformers and Importing Dependencies

In [1]:
!pip install transformers

In [3]:
!conda install -c anaconda beautifulsoup4 -y

Solving environment: done

## Package Plan ##

  environment location: /home/simon/anaconda3/envs/nlp_becode

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.3       |     pyhb0f4dca_0          87 KB  anaconda
    soupsieve-2.0.1            |             py_0          33 KB  anaconda
    ------------------------------------------------------------
                                           Total:         120 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     anaconda/noarch::beautifulsoup4-4.9.3-pyhb0f4dca_0
  soupsieve          anaconda/noarch::soupsieve-2.0.1-py_0

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2020.12.~ --> anaconda::ca-certificates-2020.10.14-0
  certifi            conda-forge::certifi-2020.12.5-py36h5~ --

In [4]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

# 1. Load Summarization Pipeline

In [5]:
summarizer = pipeline("summarization")

# 2. Get Blog Post from Medium

In [6]:
URL = "https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"
#URL = "https://hackernoon.com/will-the-game-stop-with-gamestop-or-is-this-just-the-beginning-2j1x32aa"

In [7]:
r = requests.get(URL)

In [8]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [13]:
ARTICLE

"Sign in A Bayesian Take On Model Regularization Ryan Sander Feb 1·10 min read I’m currently reading “How We Learn” by Stanislas Dehaene. First off, I cannot recommend this book enough to anyone interested in learning, teaching, or AI. One of the main themes of this book is explaining the neurological and psychological bases of why humans are so good at learning things quickly and with great sample-efficiency, i.e. given only a limited amount of experience¹. One of Dehaene’s main arguments of why humans can learn so effectively is because we are able to reduce the complexity of models we formulate of the world. In accordance with the principle of Occam’s Razor², we find the simplest model possible that explains the data we experience, rather than opting for more complicated models. But why do we do this, even from birth¹? One argument is that, contrary to the frequentist view in child psychology (the belief that babies learn solely through their experiences), we are already imparted wi

In [6]:
ARTICLE

'Will The Game Stop with Gamestop Or Is This Just The Beginning? Crypto, Markets, Trading The GameStop squeeze on short-sellers is an extraordinary event in markets, where at face value, retail traders and investors have worked together in an attempt to put some of the largest wall street institutions out of business. The events can be interpreted with many viable lenses and there are ironies baked in that are pure serendipity. There has been a centrally controlled game in the global financial system in which insiders benefited while outsiders got hurt that comes to a head with a company called GameStop. The broking firm of most of the retail side of this warfare ‘RobinHood’ is literally stealing from its poor, retail investors to give to its rich, capital backers. One of the historical realities of this game has been that macro-investing – the sages of not only portfolio management, but often also sophisticated social and cultural figures – have had a hard time making money in markets

# 3. Chunk Text

In [21]:
max_chunk_length = 500

#### end of sentence tag to split on

In [22]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [23]:
sentences = ARTICLE.split('<eos>')

In [24]:
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk_length:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        chunks.append(sentence.split(' '))
        
for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

In [25]:
len(chunks)

5

# 4. Summarize Text

In [31]:
chunks

['Sign in A Bayesian Take On Model Regularization Ryan Sander Feb 1·10 min read I’m currently reading “How We Learn” by Stanislas Dehaene.   First off, I cannot recommend this book enough to anyone interested in learning, teaching, or AI.   One of the main themes of this book is explaining the neurological and psychological bases of why humans are so good at learning things quickly and with great sample-efficiency, i.  e.   given only a limited amount of experience¹.   One of Dehaene’s main arguments of why humans can learn so effectively is because we are able to reduce the complexity of models we formulate of the world.   In accordance with the principle of Occam’s Razor², we find the simplest model possible that explains the data we experience, rather than opting for more complicated models.   But why do we do this, even from birth¹?   One argument is that, contrary to the frequentist view in child psychology (the belief that babies learn solely through their experiences), we are al

In [28]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [29]:
res[0]

{'summary_text': ' In machine learning, regularization is an essential and common practice to ensure that a model attains high out-of-sample performance . We can do this by placing a prior on our distribution of models . We will focus on analytical regularization techniques for supervised learning .'}

In [30]:
' '.join([summ['summary_text'] for summ in res])

" In machine learning, regularization is an essential and common practice to ensure that a model attains high out-of-sample performance . We can do this by placing a prior on our distribution of models . We will focus on analytical regularization techniques for supervised learning .  These regularization techniques accomplish different objectives for controlling the complexity of our models . In the next section, we will analyze these claims for regression problems, but they extend to other supervised learning tasks, such as classification . We’ll focus on rigorously presenting the mathematics behind these claims .  We’ll now examine a similar case with a Laplace prior . Placing a Gaussian prior on our parameters is equivalent to regularizing our parameters with an L2-norm penalty . The significance of the above derivations may be best understood in the context of “How We Learn”  We briefly introduced some common analytical, supervised regularization techniques (Ridge, Lasso, and Elast

In [32]:
text = ' '.join([summ['summary_text'] for summ in res])

# 5. Output to Text File

In [90]:
with open('blogsummary.txt', 'w') as f:
    f.write(text)

In [34]:
List = [['a', 'b', 'c'], ['a', 'b', 'c'], ['a', 'b', 'c']]

In [35]:
'1'.join(['2'.join(l) for l in List])

'a2b2c1a2b2c1a2b2c'